In [1]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from dotenv import load_dotenv

# Get the current working directory
current_dir = os.getcwd()

# Append the parent directory
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

# ignore warrnings
import warnings
warnings.filterwarnings("ignore")
from db.dbConnections import PostgresConnection
from scripts.userSatisfactionAnalysis import *
from scripts.utils import *

In [2]:
 # Establish database connection
db = PostgresConnection()
db.connect()

# Define the query to get data
query = 'SELECT * FROM xdr_data'


Connected to PostgreSQL database


In [3]:
# Load and clean data from the database
df = load_and_clean_data_from_db(db, query)
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,Thu Apr 04 2019 15:01:18 GMT+0300 (East Africa...,770.0,Thu Apr 25 2019 17:35:31 GMT+0300 (East Africa...,662.0,1.823653e+09,2.082014e+14,3.366496e+10,3.552121e+13,9164566995485190,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,Tue Apr 09 2019 16:04:04 GMT+0300 (East Africa...,235.0,Thu Apr 25 2019 11:15:48 GMT+0300 (East Africa...,606.0,1.365104e+09,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,Tue Apr 09 2019 20:42:11 GMT+0300 (East Africa...,1.0,Thu Apr 25 2019 14:58:13 GMT+0300 (East Africa...,652.0,1.361763e+09,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,Wed Apr 10 2019 03:31:25 GMT+0300 (East Africa...,486.0,Thu Apr 25 2019 10:36:35 GMT+0300 (East Africa...,171.0,1.321510e+09,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,Fri Apr 12 2019 23:10:23 GMT+0300 (East Africa...,565.0,Thu Apr 25 2019 13:40:32 GMT+0300 (East Africa...,954.0,1.089009e+09,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
 # Aggregate data
aggregated_data = aggregate_data(df)

In [5]:
# Perform K-Means clustering for experience
experience_clusters, kmeans_experience = perform_kmeans_clustering(aggregated_data.iloc[:, 1:], 3)
aggregated_data['Experience_Cluster'] = experience_clusters
experience_cluster_means = aggregated_data.groupby('Experience_Cluster').mean()


In [6]:
# Perform K-Means clustering for engagement
engagement_clusters, kmeans_engagement = perform_kmeans_clustering(aggregated_data.iloc[:, 1:], 3)
aggregated_data['Engagement_Cluster'] = engagement_clusters


In [7]:
 # Identify worst experience and less engaged clusters
worst_experience_cluster = experience_cluster_means[['Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']].idxmax().iloc[0]
less_engaged_cluster = 0  # Assuming 0 is the less engaged cluster


In [8]:
columns_of_interest = [
    'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 
    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'
]
kmeans_engagement = KMeans(n_clusters=3)
kmeans_engagement.fit(aggregated_data[columns_of_interest])
print("Shape of aggregated_data:", aggregated_data[columns_of_interest].shape)
print("Shape of engagement_cluster_centers:", kmeans_engagement.cluster_centers_.shape)


Shape of aggregated_data: (107265, 6)
Shape of engagement_cluster_centers: (3, 6)


In [9]:
# Calculate engagement and experience scores
aggregated_data = calculate_scores(aggregated_data, experience_cluster_means, kmeans_engagement.cluster_centers_, less_engaged_cluster, worst_experience_cluster)


In [10]:
# Compute satisfaction score
aggregated_data = compute_satisfaction_score(aggregated_data)

# Get top 10 satisfied customers
top_10_satisfied = aggregated_data.nlargest(10, 'Satisfaction_Score')
print("Top 10 satisfied customers:")
print(top_10_satisfied)


Top 10 satisfied customers:
               IMSI  TCP DL Retrans. Vol (Bytes)  TCP UL Retrans. Vol (Bytes)  \
76093  2.082017e+14                 4.289488e+09                   11989838.0   
70098  2.082017e+14                 4.288060e+09                      60496.0   
25608  2.082010e+14                 4.268432e+09                     215354.0   
27762  2.082014e+14                 4.254644e+09                      15816.0   
82401  2.082018e+14                 4.211189e+09                      68802.0   
66329  2.082017e+14                 4.166591e+09                       3989.0   
31005  2.082014e+14                 4.131046e+09                    6891979.0   
81324  2.082018e+14                 4.117753e+09                      51555.0   
76790  2.082017e+14                 3.968072e+09                      48174.0   
97867  2.082019e+14                 3.785295e+09                    1576179.0   

       Avg RTT DL (ms)  Avg RTT UL (ms)  Avg Bearer TP DL (kbps)  \
76093       

In [11]:
# Build regression model
reg_model = build_regression_model(aggregated_data)
print("Regression coefficients:", reg_model.coef_)

Regression coefficients: [0.5 0.5]


In [12]:
# Perform K-Means clustering on satisfaction scores
aggregated_data, kmeans_satisfaction = cluster_satisfaction_scores(aggregated_data)

# Aggregate cluster summary
cluster_summary = aggregate_cluster_summary(aggregated_data)
print("Cluster summary:")
print(cluster_summary)


Cluster summary:
                      Engagement_Score  Experience_Score  Satisfaction_Score
Satisfaction_Cluster                                                        
0                         1.023465e+07      1.027366e+07        1.025415e+07
1                         2.198578e+09      2.198799e+09        2.198689e+09


In [15]:
print(aggregated_data.columns)


Index(['IMSI', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)',
       'Avg Bearer TP UL (kbps)', 'Experience_Cluster', 'Engagement_Cluster',
       'Engagement_Score', 'Experience_Score', 'Satisfaction_Score',
       'Satisfaction_Cluster'],
      dtype='object')


In [16]:
db_config = {
    'user': 'etech',
    'password': 'etech',
    'host': 'localhost',
    'port': 5432,
    'database': 'telecom_db'
}
user_score_data = aggregated_data[['IMSI', 'Engagement_Score', 'Experience_Score', 'Satisfaction_Score']]
export_to_postgresql(user_score_data, 'user_metrics', db_config)


Data exported successfully to the table 'user_metrics' in the PostgreSQL database.
